In [2]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import matplotlib.pyplot as plt
import math
from numpy.random import choice
from pathlib import Path
import scipy.interpolate
from FEM_TPT import *
import scipy.stats
pitorch = torch.Tensor([math.pi])


def dU(x):
    beta = torch.tensor([[-1]])
    alpha = torch.tensor([[1]])
    dUx = x*(beta + alpha * x**2)
    
    return dUx

def dU_control(beta,gamma,q,gradq):
    control = torch.tensor(2)*gamma/beta*(gradq[:,1]/q)
    
    return control



In [3]:
class Model1(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)
        # output layer
        self.linear2 = nn.Linear(hidden_size, out_size)
        
    def forward(self, xb):
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        tanhf = nn.Tanh()
        out = tanhf(out)
        # Get predictions using output layer
        out = self.linear2(out)
        # apply activation function again
        out = torch.sigmoid(out)
        return out
    

In [4]:
class Model2(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)
        # output layer
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, out_size)
        
    def forward(self, xb):
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        tanhf = nn.Tanh()
        out = tanhf(out)
        # Get predictions using output layer
        out = self.linear2(out)
        out = tanhf(out)
        out = self.linear3(out)
        # apply activation function again
        out = torch.sigmoid(out)
        return out

In [5]:
class Model3(nn.Module):
    """Feedfoward neural network with 3 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)
        # output layer
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size,hidden_size)
        self.linear4 = nn.Linear(hidden_size, out_size)
        
    def forward(self, xb):
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        tanhf = nn.Tanh()
        out = tanhf(out)
        # Get predictions using output layer
        out = self.linear2(out)
        out = tanhf(out)
        out = self.linear3(out)
        out = tanhf(out)
        out = self.linear4(out)
        # apply activation function again
        out = torch.sigmoid(out)
        return out

In [7]:

# model = torch.load('Duffiing_gamma0.5_beta10_PINN_EM.pt') 
model = torch.load('./data/Duffiing_gamma0.5_beta20_PINN_uniform.pt') 
model.eval()


Model1(
  (linear1): Linear(in_features=2, out_features=40, bias=True)
  (linear2): Linear(in_features=40, out_features=1, bias=True)
)

In [8]:
def get_init_pt(circle_r):
    # center of the circle (x, y)
    circle_x = -1
    circle_y = 0

    # random angle
    alpha = 2 * math.pi * np.random.random()
    # random radius
    r = circle_r * math.sqrt(np.random.random())
    # calculating coordinates
    x = r * math.cos(alpha) + circle_x
    y = r * math.sin(alpha) + circle_y
    
    return torch.tensor([[x]]),torch.tensor([[y]])

In [9]:
def get_init_pt_weighted(r,t,q_circ):
    # center of the circle (x, y)
    circle_x = -1
    circle_y = 0
    alpha = choice(t, 1, p=q_circ.detach().numpy().squeeze())

    # calculating coordinates
    x = r * math.cos(alpha) + circle_x
    y = r * math.sin(alpha) + circle_y
    
    return torch.tensor([[x]]),torch.tensor([[y]])

In [10]:
def get_init_pt_weighted_ellipse(t,rx,ry,q_circ):
    # center of the circle (x, y)
    circle_x = -1
    circle_y = 0
    alpha = choice(t, 1, p=q_circ.detach().numpy().squeeze())

    # calculating coordinates
    x = rx*math.cos(alpha)+circle_x
    y = ry*math.sin(alpha)+ circle_y
    
    return torch.tensor([[x]]),torch.tensor([[y]])

def get_init_pt_ellipse(t,rx,ry,q_circ):
    # center of the circle (x, y)
    circle_x = -1
    circle_y = 0
    alpha = 2 * math.pi * np.random.random()

    # calculating coordinates
    x = rx*math.cos(alpha)+circle_x
    y = ry*math.sin(alpha)+ circle_y
    
    return torch.tensor([[x]]),torch.tensor([[y]])


In [11]:
def running_traj(Time,M,beta,gamma,delt):
    print(Time)
    I = (Time+100)*torch.ones(M)
    rsquare = torch.tensor([0.3]).pow(2)
    rx = 0.3
    ry = 0.4
    t = np.linspace(0,2*np.pi, 200)
    x_circ = -1 + rx*np.cos(t)
    y_circ = 0 + ry*np.sin(t)
    xy_circ = torch.vstack((torch.tensor(x_circ),torch.tensor(y_circ))).T
    q_circ = model(xy_circ.float())
    q_circ = q_circ/torch.sum(q_circ)
    a = torch.tensor([-1,0])
    b = torch.tensor([1,0])
    for m in range(M):
        x0, y0 = get_init_pt_weighted_ellipse(t,rx,ry,q_circ)
        
        print(m)
        newX = x0
        newY = y0

        w = torch.randn(Time)
        w = torch.sqrt(delt)*w
        
        X = torch.tensor([])
        Y = torch.tensor([])
        
        X = torch.cat((X,x0), 0)
        Y = torch.cat((Y,y0), 0)
        
        
        inA = False
        lastA = 0

        for i in range(Time):

            newR = torch.cat((newX, newY), 1)

            newR.requires_grad_(True)

            dUx = dU(newX)
            Q = model(newR)
            derivQ = torch.autograd.grad(Q,newR,allow_unused=True, retain_graph=True, \
                                    grad_outputs = torch.ones_like(Q), create_graph=True)[0]

            dU_control_y = dU_control(beta,gamma,Q,derivQ)

            newX = newX + newY * delt
            newY = newY - (gamma*newY + dUx - dU_control_y)*delt + torch.sqrt(2*gamma/beta)*w[i]
            
            X = torch.cat((X,newX), 0)
            Y = torch.cat((Y,newY), 0)
            
            distA = (newX - a[0]).pow(2)/(rx**2) + (newY - a[1]).pow(2)/(ry**2)
            if distA <= 1.0:
                inA = True
            else:
                if inA == True:
                    lastA = i
                inA = False
            if inA == False:
                distB = (newX - b[0]).pow(2)/(rx**2) + (newY - b[1]).pow(2)/(ry**2)
                if distB <= 1.0:
                    I[m] = i - lastA
                    print('TAB: ', I[m])

                    break
            
    return I
            
def mean_confidence_interval(data, confidence=0.95):
    n = len(data)
    m, se = np.mean(data), scipy.stats.sem(data)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [12]:
# betaT = torch.tensor(10)
# delt = torch.tensor(0.005)
# gamma = torch.tensor(0.5)
# I = running_traj(20000,500,betaT,gamma,delt)

In [13]:
# torch.save(I,'I_beta10_250.pt')

# transition rate

In [14]:
# Original potential function
class functions:
    def __init__(self,beta,alpha):
        self.beta = beta
        self.alpha = alpha
        
    def funU(self,x):
        U = 0.5*self.beta*(x**2) + 1/4*self.alpha*(x**4)

        return U

    def funT(self,p):
        T = 0.5*(p**2)
        return T

    def dU(self,x):
        dUx = x*(self.beta + self.alpha * (x**2))

        return dUx
    
def Hamiltonian(x,y):
    return 0.5*y**2 + 0.25*x**4 - 0.5*x**2
def fpot(x):
    return Hamiltonian(x[:,0],x[:,1])
def divfree(x,y):
    f1 = y
    f2 = -x**3 + x
    return f1,f2
    

In [16]:
model_10 = torch.load('./data/Duffiing_gamma0.5_beta10_PINN_EM.pt')
model_10.eval()

model_20 = torch.load('./data/Duffiing_gamma0.5_beta20_PINN_uniform.pt')
model_20.eval()

Model1(
  (linear1): Linear(in_features=2, out_features=40, bias=True)
  (linear2): Linear(in_features=40, out_features=1, bias=True)
)

In [17]:
"""
beta = 10
"""
beta = 10.0
gamma = 0.5
data_folder = Path('./Duffing_beta10')

pts_file = data_folder/'Duffing_pts_ellip.csv'
tri_file = data_folder/'Duffing_tri_ellip.csv'
q_file = data_folder/'Duffing_committor_beta10_ellip.csv'
qminus_file = data_folder/'Duffing_backward_committor_beta10_ellip.csv'
Apts_file = data_folder/'Apts_ellip.csv'
Bpts_file = data_folder/'Bpts_ellip.csv'
Atri_file = data_folder/'Atri_ellip.csv'
Btri_file = data_folder/'Btri_ellip.csv'


data_pts = np.loadtxt(pts_file, delimiter=',', dtype=float)
data_q = np.loadtxt(q_file, delimiter=',', dtype=float)
data_qminus = np.loadtxt(qminus_file,delimiter=',', dtype=float)
data_pts_minus = np.hstack((data_pts[:,0][:,None],-data_pts[:,1][:,None]))
data_tri = np.loadtxt(tri_file, delimiter=',', dtype=int)
# NO NEED TO CONVERT TO TORCH TENSOR
A_pts = np.loadtxt(Apts_file, delimiter=',', dtype=float)
B_pts = np.loadtxt(Bpts_file, delimiter=',', dtype=float)
A_tri = np.loadtxt(Atri_file, delimiter=',', dtype=int)
B_tri = np.loadtxt(Btri_file, delimiter=',', dtype=int)

A_pts_tensor = torch.tensor(A_pts, dtype = torch.float32)
B_pts_tensor = torch.tensor(B_pts, dtype = torch.float32)

pts_beta10 = torch.tensor(data_pts,dtype = torch.float32)
pts_beta10_minus = torch.tensor(data_pts_minus,dtype = torch.float32)
q_fem_beta10 = torch.tensor(data_q[:,2],dtype = torch.float32)
qminus_fem_beta10 = torch.tensor(data_qminus[:,2],dtype = torch.float32)

pts_beta10.requires_grad_(True)
Funs = functions(torch.tensor(-1),torch.tensor(1))
Q = model_10(pts_beta10)
Q_minus = torch.tensor(1) - model_10(pts_beta10_minus)

beta_tensor = torch.tensor(10)
U = Funs.funU(pts_beta10[:,0]) # potential energy function
T = Funs.funT(pts_beta10[:,1]) # kinetic energy function

Z_10 = invariant_pdf(data_pts,data_tri,A_pts,A_tri,B_pts,B_tri,fpot,beta)

Rcurrent, Rrate_10 = reactive_current_transition_rate_Langevin(data_pts,data_tri,fpot,divfree,beta, 0.5
                                                            ,Q.detach().numpy(),Q_minus.detach().numpy(),Z_10)

rho_A_10 = probability_last_A_Langevin(data_pts,data_tri,A_pts,A_tri,fpot,beta,Q.detach().numpy(),Q_minus.detach().numpy(),Z_10)
rho_AB_10 = probability_reactive_Langevin(data_pts,data_tri,fpot,beta,Q.detach().numpy(),Q_minus.detach().numpy(),Z_10)

print("""BY NN and FEM functions: rho_A is {},  rho_AB is {},
      escape rate is {}, transition rate is {}""".format(rho_A_10, rho_AB_10,\
                                                          Rrate_10/rho_A_10, Rrate_10))



BY NN and FEM functions: rho_A is [0.37091199],  rho_AB is [0.03969082],
      escape rate is [0.01220015], transition rate is [0.00452518]


In [18]:
# """
# beta = 20
# """
beta = 20.0
data_folder = Path('./Duffing_beta20')

pts_file = data_folder/'Duffing_pts_ellip.csv'
tri_file = data_folder/'Duffing_tri_ellip.csv'
q_file = data_folder/'Duffing_committor_beta20_ellip.csv'
qminus_file = data_folder/'Duffing_backward_committor_beta20_ellip.csv'
Apts_file = data_folder/'Apts_ellip.csv'
Bpts_file = data_folder/'Bpts_ellip.csv'
Atri_file = data_folder/'Atri_ellip.csv'
Btri_file = data_folder/'Btri_ellip.csv'

data_pts = np.loadtxt(pts_file, delimiter=',', dtype=float)
data_q = np.loadtxt(q_file, delimiter=',', dtype=float)
data_qminus = np.loadtxt(qminus_file,delimiter=',', dtype=float)
data_pts_minus = np.hstack((data_pts[:,0][:,None],-data_pts[:,1][:,None]))
data_tri = np.loadtxt(tri_file, delimiter=',', dtype=int)
# NO NEED TO CONVERT TO TORCH TENSOR
A_pts = np.loadtxt(Apts_file, delimiter=',', dtype=float)
B_pts = np.loadtxt(Bpts_file, delimiter=',', dtype=float)
A_tri = np.loadtxt(Atri_file, delimiter=',', dtype=int)
B_tri = np.loadtxt(Btri_file, delimiter=',', dtype=int)

A_pts_tensor = torch.tensor(A_pts, dtype = torch.float32)
B_pts_tensor = torch.tensor(B_pts, dtype = torch.float32)

pts_beta20 = torch.tensor(data_pts,dtype = torch.float32)
pts_beta20_minus = torch.tensor(data_pts_minus,dtype = torch.float32)
q_fem_beta20 = torch.tensor(data_q[:,2],dtype = torch.float32)
qminus_fem_beta20 = torch.tensor(data_qminus[:,2],dtype = torch.float32)

pts_beta20.requires_grad_(True)
Funs = functions(torch.tensor(-1),torch.tensor(1))
Q = model_20(pts_beta20)
Q_minus = torch.tensor(1) - model_20(pts_beta20_minus)

U = Funs.funU(pts_beta20[:,0]) # potential energy function
T = Funs.funT(pts_beta20[:,1]) # kinetic energy function

Z_20 = invariant_pdf(data_pts,data_tri,A_pts,A_tri,B_pts,B_tri,fpot,beta)

Rcurrent, Rrate_20 = reactive_current_transition_rate_Langevin(data_pts,data_tri,fpot,divfree,beta, 0.5
                                                            ,Q.detach().numpy(),Q_minus.detach().numpy(),Z_20)

rho_A_20 = probability_last_A_Langevin(data_pts,data_tri,A_pts,A_tri,fpot,beta,Q.detach().numpy(),Q_minus.detach().numpy(),Z_20)
rho_AB_20 = probability_reactive_Langevin(data_pts,data_tri,fpot,beta,Q.detach().numpy(),Q_minus.detach().numpy(),Z_20)

print("""BY NN and FEM functions: rho_A is {},  rho_AB is {},
      escape rate is {}, transition rate is {}""".format(rho_A_20, rho_AB_20,\
                                                          Rrate_20/rho_A_20, Rrate_20))


BY NN and FEM functions: rho_A is [0.28679121],  rho_AB is [0.00424755],
      escape rate is [0.00164725], transition rate is [0.00047242]


# Calculate mean escape time from A using results by optimal control & NN

Case for beta = 10

In [21]:
I = torch.load('./data/I_beta10_250_ellip.pt')
delt = torch.tensor(0.005)
# with open('I_duffing_controlled_beta1','rb') as f:
#     I = pickle.load(f)
average_time_steps, left, right = mean_confidence_interval(I.detach().numpy())
average_time = average_time_steps*delt.detach().numpy()
estimated_rate = 1/average_time
print('E_tau_AB: {}'.format(average_time))

E_tau_AB: 6.880499839782715


In [22]:
# average_time = 6.77
escape_rate_10 = 1/(average_time*(rho_A_10/rho_AB_10))
print('escape rate for beta = 10 is: {}'.format(escape_rate_10))
transition_rate_10 = rho_AB_10/average_time
print('transition rate for beta = 10 is: {}'.format(transition_rate_10))
conf_left = rho_AB_10/(right*delt.detach().numpy())
conf_right = rho_AB_10/(left*delt.detach().numpy())
print('confidence interval transition rate for beta = 10 is: [{},{}]'.format(conf_left,conf_right))

escape rate for beta = 10 is: [0.01555246]
transition rate for beta = 10 is: [0.0057686]
confidence interval transition rate for beta = 10 is: [[0.00549516],[0.00607067]]


In [23]:
print(left*delt - average_time_steps*delt)
print(right*delt - average_time_steps*delt)

tensor(-0.3424)
tensor(0.3424)


beta = 20

In [24]:
I = torch.load('./data/I_beta20_250_ellipse.pt')
delt = torch.tensor(0.005)
average_time_steps, left, right = mean_confidence_interval(I.detach().numpy())
average_time = average_time_steps*delt.detach().numpy()
estimated_rate = 1/average_time
print('E_tau_AB: {}'.format(average_time))

E_tau_AB: 7.33843994140625


In [25]:
escape_rate_20 = 1/(average_time*(rho_A_20/rho_AB_20))
print('escape rate for beta = 1 is: {}'.format(escape_rate_20))
transition_rate_20 = rho_AB_20/average_time
print('transition rate for beta = 1 is: {}'.format(transition_rate_20))
conf_left = rho_AB_20/(right*delt.detach().numpy())
conf_right = rho_AB_20/(left*delt.detach().numpy())
print('confidence interval transition rate for beta = 1 is: [{},{}]'.format(conf_left,conf_right))

escape rate for beta = 1 is: [0.00201822]
transition rate for beta = 1 is: [0.00057881]
confidence interval transition rate for beta = 1 is: [[0.00055362],[0.0006064]]


# plot trajectories

In [31]:
def plot_traj(Time,M,beta,gamma,delt):
    print(Time)
    I = (Time+100)*torch.ones(M)
    rsquare = torch.tensor([0.3]).pow(2)
    rx = 0.3
    ry = 0.4
    t = np.linspace(0,2*np.pi, 200)
    x_circ = -1 + rx*np.cos(t)
    y_circ = 0 + ry*np.sin(t)
    xy_circ = torch.vstack((torch.tensor(x_circ),torch.tensor(y_circ))).T
    q_circ = model_20(xy_circ.float())
    q_circ = q_circ/torch.sum(q_circ)
    
    a = torch.tensor([-1,0])
    b = torch.tensor([1,0])
    
    for m in range(M):
#         x0,y0 = get_init_pt_weighted(0.3,t,q_circ)
        x0, y0 = get_init_pt_weighted_ellipse(t,rx,ry,q_circ)
        
        print(m)
        newX = x0
        newY = y0
        newX_orig = x0
        newY_orig = y0

        w = torch.randn(Time)
        w = torch.sqrt(delt)*w
        
        X = torch.tensor([])
        Y = torch.tensor([])
        
        X = torch.cat((X,x0), 0)
        Y = torch.cat((Y,y0), 0)
        
        X_orig = torch.tensor([])
        Y_orig = torch.tensor([])
        
        X_orig = torch.cat((X_orig,x0), 0)
        Y_orig = torch.cat((Y_orig,y0), 0)
        
        
        flag = 0

        for i in range(Time):

            b = torch.tensor([1,0])

            newR = torch.cat((newX, newY), 1)

            newR.requires_grad_(True)

            dUx = dU(newX)
            Q = model(newR)
            derivQ = torch.autograd.grad(Q,newR,allow_unused=True, retain_graph=True, \
                                    grad_outputs = torch.ones_like(Q), create_graph=True)[0]

            dU_control_y = dU_control(beta,gamma,Q,derivQ)

            newX = newX + newY * delt
            newY = newY - (gamma*newY + dUx - dU_control_y)*delt + torch.sqrt(2*gamma/beta)*w[i]
            
            X = torch.cat((X,newX), 0)
            Y = torch.cat((Y,newY), 0)
            
            dUx_orig = dU(newX_orig)
            
            newX_orig = newX_orig + newY_orig * delt
            newY_orig = newY_orig - (gamma*newY_orig + dUx_orig)*delt + torch.sqrt(2*gamma/beta)*w[i]
            
            X_orig = torch.cat((X_orig,newX_orig), 0)
            Y_orig = torch.cat((Y_orig,newY_orig), 0)
            
            if flag == 0:
                distB = (newX - b[0]).pow(2) + (newY - b[1]).pow(2)
                if distB < rsquare:
                    flag = 1
#                     print('we break rb = 0.3 at: ', i)
            if flag == 1:
                distB = (newX - b[0]).pow(2) + (newY - b[1]).pow(2)
                if distB < torch.tensor([0.15]).pow(2):
                    flag = 2
#                     print('we break rb = 0.15 at: ', i)
            
        if m == 0:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'b', s = 1,label = 'trajectory 1')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'b', s = 1,label = 'trajectory 1')
        if m == 1:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'g', s = 1,label = 'trajectory 2')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'g', s = 1,label = 'trajectory 2')
        if m == 2:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'orange', s = 1,label = 'trajectory 3')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'orange', s = 1,label = 'trajectory 3')
    
    t = np.linspace(0,2*np.pi,200)
    plt.figure(1)
    plt.scatter(a[0],a[1])
    plt.scatter(b[0],b[1])
    Rx = 0.3
    Ry = 0.4
    a = [-1,0]
    b = [1,0]
    plt.plot(Rx*np.cos(t)+a[0],Ry*np.sin(t)+a[1])
    plt.plot(Rx*np.cos(t)+b[0],Ry*np.sin(t)+b[1])
    plt.xlim([-2,2])
    plt.ylim([-2,2])
    plt.legend()
    plt.xlabel('$x$')
    plt.ylabel('$p$')
    plt.rcParams["figure.figsize"] = (5,5)
    plt.savefig('./data/controlled_duffing_beta20_ellip.pdf')
    
    plt.figure(2)
    plt.scatter(a[0],a[1])
    plt.scatter(b[0],b[1])
    
    Rx = 0.3
    Ry = 0.4
    a = [-1,0]
    b = [1,0]
    plt.plot(Rx*np.cos(t)+a[0],Ry*np.sin(t)+a[1])
    plt.plot(Rx*np.cos(t)+b[0],Ry*np.sin(t)+b[1])
    
    plt.xlim([-2,2])
    plt.ylim([-2,2])
    plt.legend()
    plt.xlabel('$x$')
    plt.ylabel('$p$')
    plt.rcParams["figure.figsize"] = (5,5)
    plt.savefig('./data/uncontrolled_duffing_beta20_ellip.pdf')
    
    return I
                


In [34]:
# beta = torch.tensor(20)
# delt = torch.tensor(0.005)
# gamma = torch.tensor(0.5)
# I = plot_traj(5000,3,beta,gamma,delt)